In [1]:
import os
import requests
import csv
import random

from dotenv import load_dotenv
from klaviyo_api import KlaviyoAPI

from langchain.prompts import PromptTemplate
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import (RunnablePassthrough, RunnableLambda, RunnableParallel)

In [2]:
_ = load_dotenv()

In [3]:
# Extract Klaviyo (private) API key from .env file
klaviyo_key='pk_a7344fe3a27b6f571d6873734a388ad807'

In [4]:
# Instantiate the Klaviyo Python API with the API key

klaviyo = KlaviyoAPI(klaviyo_key, max_delay=60, max_retries=3, test_host=None)

In [5]:
# Extract profile data and events for a Klaviyo ID as a string we can feed to LangChain

def get_customer_data(custid):
    profile=klaviyo.Profiles.get_profile(custid)
    data = profile['data']
    attributes = data['attributes']
    properties = attributes['properties']
    location = attributes['location']
    info = {key: value for key, value in attributes.items() if key not in ['properties', 'location']}
    combined_dict = {**properties, **location, **info}
    return combined_dict

In [6]:
grab_attention = ChatPromptTemplate.from_template(
    '''
Write a short 2 line introduction to an email with a hook from Max's Store to {first_name}. The email should be pitching the release and special offer of a new line of shoes called the MegaShoe 1s. Write it according to the following rules:
- Infer gender and age from {first_name} and tailor your style to who you're addressing
- If the customer has a high {clv}, offer them a better deal than a customer with a lower {clv}
- If a customer has a high {churn_risk} use more urgent wording in your hook
- If a customer has a higher {upsales_propensity} offer a more expensive product
- Tease the {recommended_incentive}
    
Remember not to mention these properties explicitly, but craft your message to resonate with these insights.
No emojis and keep it short and simple. Write like a 7th grader to ensure your email is easily understood by a wide audience.
Get creative, don't just fall into bland platitudes, and you don't have to follow the examples but use them to guide you.
ONLY WRITE ONE DRAFT
__

Examples: 
Hi Doug, We're excited to introduce our new product, the MegaShoe 1s, and we're offering you an exclusive 20% discount for a limited time as one of our valued customers!
Hi Debby, Get ready to upgrade your style with our latest innovation - the MegaShoe 1s Supreme! As a loyal customer, we're offering you a special deal of buy one, get one 50% off for a limited time only.
Hi Kendrick, Don't miss out on our newest arrival - the MegaShoe 1s Pro! With its advanced features and sleek design, it's a must-have for anyone on-the-go. And as a valued customer, we're giving you an exclusive 30% off your first purchase.

'''
)

build_anticipation = ChatPromptTemplate.from_template(
    '''
Write an the body of an email from Max's Store to a customer following up on the introduction of the new MegaShoe 1s. The body should:
- Establish the customer's need for a new shoe alternative
- Emphasize the key features and benefits of the MegaShoe 1s
- Highlight the special offer and create a sense of urgency
- Encourage the customer to take action and make a purchase
- Infer gender and age from {first_name} and tailor your style to who you're addressing
- Consider the customer's {clv}, {churn_risk}, and {upsales_propensity} to tailor the tone and content
    
Remember not to mention these properties explicitly, but craft your message to resonate with these insights.
No emojis and keep it short and simple. Write like a 7th grader to ensure your email is easily understood by a wide audience.
Get creative, don't just fall into bland platitudes, and you don't have to follow the examples but use them to guide you.
DO NOT MENTION THE CUSTOMER'S NAME HERE. YOU ARE WRITING ONE BODY PARAGRAPH NOT AN INTRO OR CONCLUSION. LITERALLY JUST ONE PARAGRAPH.
ONLY WRITE ONE DRAFT
__

Examples:
- We've just launched our revolutionary MegaShoe 1s, designed to keep you comfortable and confident all day long. Whether you're hitting the gym or running errands, these shoes have got you covered. Here's what you need to know: Comfort and support: Soft materials and arch support for happy feet. Stylish and versatile: Cute design that looks great for any occasion. Durable and long-lasting: Built to withstand your active lifestyle. 
- Sick of wearing out your favorite kicks too fast? We feel you! That's why we created the MegaShoe 1s - the ultimate shoe for the active dude on the go! These bad boys are packed with: Comfy tech: Cushioning and breathability for happy feet. Supportive vibes: Stability features for high-impact activities. Fresh style: Sleek design that's perfect for the gym or just chillin'.
- Are you tired of wearing out your favorite shoes too quickly? Do you need a new go-to pair that can keep up with your active lifestyle? We've got you covered! Our brand-new MegaShoe 1s are designed to provide unparalleled comfort, support, and durability. With their cutting-edge technology and sleek design, they're perfect for fitness enthusiasts like yourself. Here's what sets them apart: Long-lasting comfort: Breathable materials and cushioning ensure your feet stay happy all day. Superior support: Stability features and arch support for high-impact activities. Style and versatility: Sleek design that looks great for both workouts and everyday wear. 
'''
)

call_to_action = ChatPromptTemplate.from_template(
    '''
Craft a compelling call to action that directs the customer on what to do next. Use actionable language that conveys urgency and benefit. Ensure the call to action is simple, direct, and easy to follow. 
- Infer gender and age from {first_name} and tailor your style to who you're addressing
- Consider the customer's {clv}, {churn_risk}, and {upsales_propensity} to tailor the tone and content
- Offer the {recommended_incentive}
- Be decisive and confident

Remember not to mention these properties explicitly, but craft your message to resonate with these insights.
No emojis and keep it short and simple. Write like a 7th grader to ensure your email is easily understood by a wide audience.
Get creative, don't just fall into bland platitudes, and you don't have to follow the examples but use them to guide you.
DO NOT MENTION THE CUSTOMER'S NAME HERE.
ONLY WRITE ONE DRAFT
__   

Example:
- Don't Miss Out! Grab 10% off your first MegaShoe 1s purchase and get the comfort and support you need. Limited time offer - Shop Now and Get Moving! Best Regards, Your Friends at Max's Store
- Treat Yourself to the Best! Enjoy 12% off your first MegaShoe 1s purchase and discover the perfect blend of style and comfort. Shop Now and Elevate Your Everyday! Sincerely, your friends at Max's Store
- Upgrade Your Shoe Game Now! Get 15% off your first MegaShoe 1s purchase and experience the ultimate in comfort and performance. Shop Now and Take Your Fitness to the Next Level! Warmly, your friends at Max's Store
    '''
)

subject_line = ChatPromptTemplate.from_template(
    '''
Write a short but eye-catching subject line to an email from Max's Store mentioning the customer by {first_name} and the {recommended_incentive}.
- No emojis and keep it short and simple. Write like a 7th grader to ensure your email is easily understood by a wide audience.
- Get creative, don't just fall into bland platitudes, and you don't have to follow the examples but use them to guide you.
- ONLY WRITE ONE DRAFT
__

Examples:
- Billy, 24 Hours to Claim Your Exclusive Gift!
- Hey Megan, We've Got a Shoe-tastic Deal for You! Coupon Inside!
- AJ, Unlock Your VIP Access: Free Shipping on MegaShoe 1s Awaits
    '''
)

In [7]:
# Create the chat model:
model = ChatOpenAI(model_name="gpt-4", temperature=0.2)

# Create the sub-chains:
grab_attention_chain = (grab_attention | model | StrOutputParser())

build_anticipation_chain = (build_anticipation | model | StrOutputParser())

call_to_action_chain = (call_to_action | model | StrOutputParser())

subject_line_chain = (subject_line | model | StrOutputParser())

/usr/local/anaconda3/share/jupyter/venv/py311_4130_v2/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


### Create a Function That Builds One Email from the Chains

In [8]:
# define a function that generates an email for a given customer based on their Klaviyo ID
def generate_email(custid):
    
    # Fetch customer data
    customer_data = get_customer_data(custid)
    
    # Execute sub-chains
    attention_text = grab_attention_chain.invoke(customer_data)
    anticipation_text = build_anticipation_chain.invoke(customer_data)
    action_text = call_to_action_chain.invoke(customer_data)
    subject = subject_line_chain.invoke(customer_data)

    # Combine the outputs to form the complete email
    email = f"{subject}\n\n{attention_text}\n\n{anticipation_text}\n\n{action_text}"

    return email

In [9]:
# Get profiles from Klaviyo into a dictionary
profiles = klaviyo.Profiles.get_profiles()

# Initialize an empty list to store customer IDs
cust_ids = []

# Iterate over profiles and extract customer IDs
for profile in profiles['data']:
    cust_ids.append(profile['id'])

cust_ids

['01HVFKKS3976M2BYYKRH2YE5BW',
 '01HVFKKSM9HTV1TPEASMH694MG',
 '01HVFKKSPHKNC5PV0VNN14TSJ4',
 '01HVFKKVK1HECSCYB6VVGHSSRQ',
 '01HVFMJRBBN12VGRB95NHWKBTS',
 '01HVFMJREQ54N2M732CSFX450J',
 '01HVFMJS7THFNM4R99JRWKFS1T',
 '01HVFMJSDSK7YF1K052T4S0M44',
 '01HVFMJSP8CEQK5V0YFT85VYDE',
 '01HVFMJSPXX0W8DVR30YR1T37Q',
 '01HVFMJTF6C2WHD7J4MZ5R9PAX',
 '01HVFMJTRJVBPBCY763A4M35F7',
 '01HVFMJV39QFH8AYF7D11B0KWD',
 '01HVFMJV9JTCGPE7CAYEJ6X11T']

In [10]:
def update_customer_profiles(cust_ids):
    churn_risks = ['low', 'medium', 'high']
    upsales_propensities = ['20%', '50%', '80%']
    recommended_incentives = ['gift', 'coupon', 'discount']
    clv_values = [100, 500, 1000]

    emails = {}

    for custid in cust_ids:
        custid_str = str(custid)  # Ensure the custid is a string
        # Generate random properties
        properties = {
            'clv': random.choice(clv_values),
            'churn_risk': random.choice(churn_risks),
            'upsales_propensity': random.choice(upsales_propensities),
            'recommended_incentive': random.choice(recommended_incentives)
        }

        # Prepare the payload for the API call
        payload = {
            "data": {
                "type": "profile",
                "id": custid_str,
                "attributes": {
                    'properties': properties
                }
            }
        }

        # Update the profile
        klaviyo.Profiles.update_profile(custid_str, payload)
        print(f"Updated profile for ID {custid_str} with properties: {properties}")

# Assuming cust_ids is a list of customer IDs
update_customer_profiles(cust_ids)

Updated profile for ID 01HVFKKS3976M2BYYKRH2YE5BW with properties: {'clv': 500, 'churn_risk': 'low', 'upsales_propensity': '80%', 'recommended_incentive': 'coupon'}
Updated profile for ID 01HVFKKSM9HTV1TPEASMH694MG with properties: {'clv': 500, 'churn_risk': 'low', 'upsales_propensity': '80%', 'recommended_incentive': 'gift'}
Updated profile for ID 01HVFKKSPHKNC5PV0VNN14TSJ4 with properties: {'clv': 100, 'churn_risk': 'low', 'upsales_propensity': '80%', 'recommended_incentive': 'discount'}
Updated profile for ID 01HVFKKVK1HECSCYB6VVGHSSRQ with properties: {'clv': 500, 'churn_risk': 'high', 'upsales_propensity': '80%', 'recommended_incentive': 'discount'}
Updated profile for ID 01HVFMJRBBN12VGRB95NHWKBTS with properties: {'clv': 1000, 'churn_risk': 'high', 'upsales_propensity': '20%', 'recommended_incentive': 'discount'}
Updated profile for ID 01HVFMJREQ54N2M732CSFX450J with properties: {'clv': 100, 'churn_risk': 'high', 'upsales_propensity': '50%', 'recommended_incentive': 'gift'}
Upda

In [11]:
emails = []

for customer_id in cust_ids:
    message = generate_email(customer_id)  # Generate the email from the data directly, without fetching again
    emails.append(message)
    
print(emails)


['"Russell, Your Exclusive Max\'s Store Coupon is Waiting!"\n\nHello Russell, \n\nStep into the future with our brand new MegaShoe 1s! As a valued customer, we\'re offering you an exclusive deal that\'s too good to resist. But hurry, this special offer won\'t last long!\n\nTired of shoes that just don\'t cut it? We\'ve got the perfect solution for you! Introducing our latest innovation, the MegaShoe 1s. These shoes are designed to provide ultimate comfort, support, and durability, making them perfect for your active lifestyle. They feature soft, breathable materials and arch support for all-day comfort. Plus, their stylish design makes them versatile enough for any occasion. But that\'s not all! For a limited time, we\'re offering a special deal on the MegaShoe 1s. Don\'t miss out on this opportunity to upgrade your shoe game. Act now and experience the difference of the MegaShoe 1s!\n\nAct Now! Snag a whopping 20% off your first MegaShoe 1s purchase and step into a world of unparallel

In [12]:
with open('emails.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Emails"])  # header row
    for email in emails:
        writer.writerow([email])